# Using News Data to Test Different Models

## Data preprocessing 

In [2]:
#Importing packages we need
import pandas as pd
import numpy as np
import jieba 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.svm import SVC 
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [4]:
content = pd.read_csv('sqlResult_1558435.csv', encoding = 'gb18030')

In [5]:
content.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [6]:
content[content['source'] == '新华社']

,id,author,source,content,feature,title,url
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
39,89578,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态,http://world.huanqiu.com/hot/2017-06/10866229....
95,89522,NaN,新华社,新华社韩国济州6月18日电综述：亚投行第二届年会三大亮点\r\n新华社记者 耿学鹏 严蕾\r...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",综述：亚投行第二届年会三大亮点,http://china.huanqiu.com/hot/2017-06/10866439....
113,89504,NaN,新华社,新华社北京6月18日电 经军委领导批准，《军营理论热点怎么看·2017》日前印发全军。\r\...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",《军营理论热点怎么看·2017》印发全军,http://china.huanqiu.com/hot/2017-06/10866447....
305,89312,NaN,新华社,新华社兰州6月19日电（记者张钦）记者19日了解到，刚刚出台的《甘肃省网络扶贫行动的实施方案...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",甘肃力推网络扶贫：贫困山区到2020年实现4G网络全覆盖,http://china.huanqiu.com/hot/2017-06/10866626....
312,89305,NaN,新华社,新华社乌鲁木齐6月18日电（记者杜刚、王秉阳）6月18日，新疆天山天池首次举办山地马拉松比赛...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",天山天池首次举办山地马拉松赛,http://china.huanqiu.com/hot/2017-06/10866627....
314,89303,NaN,新华社,新华社北京6月19日电（记者熊丰）记者从公安部消防局获悉，19日上午6时，由公安部消防局组织...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",9省市联动开展跨区域地震救援实战拉动演练,http://china.huanqiu.com/hot/2017-06/10866630....
543,89074,刘瑞莹,新华社,暑期旅游旺季是各大旅游景区开门迎客的好时机。然而，有5家5A级景区却高兴不起来：国家旅游...,"{""type"":""旅游"",""site"":""环球"",""commentNum"":""0"",""joi...",一年内全国12家5A级景区被国家旅游局摘牌或警告,http://go.huanqiu.com/news/2016-10/9542454.html
545,89072,NaN,新华社,新华社北京6月20日电 总部设在德国波恩的国际残奥委会在当地时间19日宣布，中国残联主席...,"{""type"":""体育"",""site"":""新浪"",""commentNum"":"" 50"",""j...",张海迪获得国际残奥委会主席参选提名,http://sports.sina.com.cn/others/others/2017-0...
651,88966,王点,新华社,新华社南京4月17日电(记者潘晔)江苏拥有不少5A景区。日前，一份来自清华大学的第三方调...,"{""type"":""旅游"",""site"":""环球"",""commentNum"":""0"",""joi...",拿什么消除游客“差评”？—江苏旅游业调查,http://go.huanqiu.com/news/2017-04/10493712.html


In [7]:
content = content.fillna('')

In [8]:
content['is_xinhua'] = np.where(content['source'].str.contains('新华'), 1, 0)

In [9]:
content['is_xinhua'].head(20)

0     0
1     0
2     0
3     1
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    1
18    0
19    0
Name: is_xinhua, dtype: int64

In [10]:
def cut(string):
    return ' '.join(jieba.cut(string))

In [11]:
#Test
cut('小狗你好')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/5d/046t421j05g25t76d5lv813w0000gn/T/jieba.cache
Loading model cost 1.054 seconds.
Prefix dict has been built succesfully.


'小狗 你好'

In [12]:
content['title'] = content['title'].fillna('')

In [13]:
content['title'] = content['title'].apply(cut)

In [14]:
content['title'].head(10)

0                  小米 MIUI   9 首批 机型 曝光 ： 共计 15 款
1           骁龙 835 在 Windows   10 上 的 性能 表现 有望 改善
2           一加 手机 5 细节 曝光 ： 3300mAh 、 充 半小时 用 1 天
3    葡 森林 火灾 造成 至少 62 人 死亡   政府 宣布 进入 紧急状态 （ 组图 ）
4         44 岁 女子 约 网友 被 拒 暴雨 中 裸奔   交警 为 其 披衣 相随
5                             金融股 一枝独秀   配置 价值 犹存
6                    配骁龙 660   全面 屏 鼻祖 夏普 新机 酝酿 中
7                          博友 早评 ： 震荡 有利 消化 套牢 筹码
8              武汉 千余 警察 出动   抓获 808 名 俊男靓女 全是 诈骗犯
9                     纳入 MSCI 指数   A股 长期 配置 价值 提升
Name: title, dtype: object

In [15]:
content = content.sample(n = 5000)

In [16]:
xinhuashe_news = content[content['source'].str.contains('新华')]

In [17]:
X_title = content['title'].values

In [18]:
X_title[:10]

array(['[ 19 ] （ 外代 二线 ） 获救 的 熊', '（ 教育 ） （ 6 ） 乡镇 幼儿园 民俗文化 迎 “ 六一 ”',
       '[ 1 ] （ 外代 二线 ） 澳大利亚 时装周 创新 时装秀',
       '王 劲松 ： 吴秀波 是 偶像   于和伟 是 演 曹操 的 第一 人',
       '[ 17 ] （ 外代 二线 ） 篮球 — — NBA 季后赛 ： 马刺 胜 火箭',
       '（ 经济 ） 给 实体 经济 划出 保护 线 — — 深圳 夯实 先进 制造业 根基 调查',
       '（ 一带 一路 · 高峰论坛 ） 中国 将 与 “ 一带 一路 ” 沿线 国家 加强 投融资 合作',
       '公鉴 ： 关于 依法 使用 新华社 新闻 产品 的 公告', '传承 红色 基因   陕北 小学 自编 特色 课间操',
       '（ 中国 网 事 ） 大 数据 时代 ， 如何 织密 个人信息 “ 保护网 ” ？'], dtype=object)

In [19]:
y = content['is_xinhua'].values

In [20]:
y[:10]

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1])

In [21]:
vectorizer = TfidfVectorizer(max_features = 20000)
X_title = vectorizer.fit_transform(X_title)

In [22]:
X_title.shape

(5000, 10813)

In [23]:
indices = np.array([i for i in range(len(y))])

In [24]:
indices

array([   0,    1,    2, ..., 4997, 4998, 4999])

In [25]:
np.random.shuffle(indices)

In [26]:
indices

array([1032, 3199, 2364, ..., 1046, 3589, 3899])

In [27]:
split_test = 0.25

In [28]:
train_indices = indices[int(len(indices) * split_test):]
test_indices = indices[:int(len(indices) * split_test)]

In [29]:
X_t_train, X_t_test, y_train, y_test = (X_title[train_indices],X_title[test_indices],y[train_indices],y[test_indices])

In [30]:
X_t_train

<3750x10813 sparse matrix of type '<class 'numpy.float64'>'
	with 27795 stored elements in Compressed Sparse Row format>

In [31]:
def get_performance(clf, X_, y_):
    y_hat = clf.predict(X_.toarray())
    print('percision is: {}'.format(precision_score(y_, y_hat)))
    print('recall is: {}'.format(recall_score(y_, y_hat)))
    print('roc_auc is: {}'.format(roc_auc_score(y_, y_hat)))
    print('confusion matrix: \n{}'.format(confusion_matrix(y_, y_hat, labels=[0, 1])))

## Try Different Models

In [ ]:
### knn

In [86]:
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X_t_train.toarray(), y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [87]:
get_performance(clf,X_t_train,y_train)

percision is: 0.953447774221705
recall is: 0.988238841978287
roc_auc is: 0.8097876237541205
confusion matrix: 
[[ 274  160]
 [  39 3277]]


In [88]:
get_performance(clf,X_t_test,y_test)

percision is: 0.9219554030874786
recall is: 0.9746146872166818
roc_auc is: 0.6777835340845313
confusion matrix: 
[[  56   91]
 [  28 1075]]


In [89]:
### Logistic Regression

In [91]:
clf = LogisticRegression()
clf.fit(X_t_train.toarray(), y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [92]:
get_performance(clf,X_t_train,y_train)

percision is: 0.8942826321467098
recall is: 1.0
roc_auc is: 0.5483870967741935
confusion matrix: 
[[  42  392]
 [   0 3316]]


In [93]:
get_performance(clf,X_t_test,y_test)

percision is: 0.8859437751004016
recall is: 1.0
roc_auc is: 0.5170068027210885
confusion matrix: 
[[   5  142]
 [   0 1103]]


In [94]:
# We can see that the logistic regression model doesn't work very well

In [95]:
### SVM

In [99]:
clf = SVC()
clf.fit(X_t_train.toarray(), y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [100]:
get_performance(clf,X_t_train,y_train)

percision is: 0.8842666666666666
recall is: 1.0
roc_auc is: 0.5
confusion matrix: 
[[   0  434]
 [   0 3316]]


In [101]:
get_performance(clf,X_t_test,y_test)

percision is: 0.8824
recall is: 1.0
roc_auc is: 0.5
confusion matrix: 
[[   0  147]
 [   0 1103]]


In [102]:
### Bayes

In [103]:
clf = GaussianNB()
clf.fit(X_t_train.toarray(), y_train)

GaussianNB(priors=None)

In [104]:
get_performance(clf,X_t_train,y_train)

percision is: 1.0
recall is: 0.9990952955367913
roc_auc is: 0.9995476477683957
confusion matrix: 
[[ 434    0]
 [   3 3313]]


In [105]:
get_performance(clf,X_t_test,y_test)

percision is: 0.9312169312169312
recall is: 0.957388939256573
roc_auc is: 0.7133883471793069
confusion matrix: 
[[  69   78]
 [  47 1056]]


In [106]:
# Kind of Overfit

In [107]:
### Random Forest

In [111]:
clf = RandomForestClassifier(max_depth = 5, n_estimators= 500)
clf.fit(X_t_train.toarray(), y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [112]:
get_performance(clf,X_t_train,y_train)

percision is: 0.8842666666666666
recall is: 1.0
roc_auc is: 0.5
confusion matrix: 
[[   0  434]
 [   0 3316]]


In [113]:
get_performance(clf,X_t_test,y_test)

percision is: 0.8824
recall is: 1.0
roc_auc is: 0.5
confusion matrix: 
[[   0  147]
 [   0 1103]]


In [32]:
clf = XGBClassifier()
clf.fit(X_t_train.toarray(), y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [33]:
get_performance(clf,X_t_train,y_train)

percision is: 0.8903520558989519
recall is: 1.0
roc_auc is: 0.5331807780320366
confusion matrix: 
[[  29  408]
 [   0 3313]]


/Users/air/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [34]:
get_performance(clf,X_t_test,y_test)

percision is: 0.8832528180354268
recall is: 1.0
roc_auc is: 0.5261437908496732
confusion matrix: 
[[   8  145]
 [   0 1097]]


/Users/air/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# 常见模型优缺点

1.knn：


优点：既可以做classification又可以做regression，理论理解容易，bias低


缺点：每一次预测都要重新计算，k值的确定比较困难，样本不平衡时会有比较大的误差，需要大量的内存


2.logistic regression:


优点：计算量小，实现较低


缺点：容易欠拟合，只能做0-1分类


3.SVM:


优点：可以解决高维问题，即大型特征空间，能够处理非线性特征的相互作用，解决小样本下机器学习问题，无需依赖整个数据； 

缺点：观测样本多的时候，效率高，对缺失数据敏感


4.Bayes:

优点：对小规模数据表现很好，对缺失数据不敏感，算法简单，计算迅速，结果可解释性较强

缺点：要预先计算概率，对输入数据的表达形式敏感


5.Random Forests：

优点：既能解决分类问题又能解决回归问题，对高维数据处理效果好，对缺失数据不敏感，可以平衡误差，训练速度快，模型泛化能力强

缺点：容易overfit，模型可解释性不强

6.XGBoost

优点：防止过拟合，支持分布式并行，对缺失值也能有较好的处理
